In [1]:
import numpy as np
import tensorflow as tf
import tokenize_uk
from gensim.models import Phrases
from nltk.tokenize import sent_tokenize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
with open('Kobzar.txt', 'r') as f: 
    data = f.read()
    sentences = sent_tokenize(data, language='russian')
    sentence_stream = [tokenize_uk.tokenize_words(x.lower()) for x in sentences]

In [52]:
phrases = Phrases(sentence_stream, min_count=1)
bigram = Phrases(phrases[sentence_stream], min_count=1, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [53]:
trigram[sentence_stream[0]]

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['кобзар',
 'шевченко',
 'тарас',
 'тарас шевченко',
 '/',
 '“',
 'кобзар',
 '”',
 'шевченко',
 'народився',
 'у',
 'селі',
 'моринці',
 'київської',
 'губ',
 '.',
 ',',
 'в',
 'родині',
 'селянина',
 '- кріпака',
 '.']

In [67]:
a = sorted(list(trigram.vocab.items()), key=lambda x: x[1], reverse=True)
counts = [(x[0].decode("utf-8"), x[1]) for x in a]

overall_count = sum([x[1] for x in counts])
probabilities = {x[0]: x[1] / float(overall_count) for x in counts}

In [63]:
probabilities[', що'] / probabilities[',']

0.031724137931034485

In [101]:
start = 'як'

def generate(start, count):
    result = []
    result.append(start)
    for i in range(count):        
        keys = [k for k in probabilities.keys() if k.startswith('{} '.format(start))]
        weights = np.array([probabilities[k] for k in keys])

        scale_factor = 1 / sum(weights) 
        weights *= scale_factor

        nxt = np.random.choice(keys, p=weights)
        result.append(' '.join(nxt.split(' ')[1:]))
        start = nxt.split(' ')[-1]
    return result 

for i in range(10): 
    print(' '.join(generate(start, 20)))

як діток старими сльозами ледве вийшла ти , треба було ?.. а я таки набрались , доглядають . - за що й
як той не прошу . — про історичну долю добуде , як була я остануся тута , де б воеводу татары убили или
як тая слава ... мати — адамові діти ! » ) , неначе злодій , де вінчати , неначе в лози колисочку
як рівний з дітками . - ха - де він мене в египетской темнице . - петербург ] послание первое святаго
як той поговір , косарал ] * * * * * * * у черниці постриглась . - сердеги . -
як я усну , гріхи . - пустиня , не пом'яне ! » ) . - запорожці , а всі упали до
як рукою махає . согласьем общим положили ті сироти . і дні минають дні і я ледве ступаю ; весело жилось ,
як квіточку , уже забува ? ” , росте марко з найпопулярніших творів періоду творчості і сіль і людям однаково ,
як на сміх людям однаково , обнімала , і стиха , то й читать ! » і їх , а щоб і пріся
як нас в люди хреститимуть , ні , серцем до вас не спіткали » ( співає : хто світ . - вихваляєш :


### lstm

In [2]:
with open('Kobzar.txt', 'r') as f: 
    lines = f.readlines()
    lines = lines[89:]
    chars = [list(l.lower()) for l in lines]
    chars = [l for l in chars if len(l) <= 100]

In [3]:
data = [item for sublist in chars for item in sublist]
n_chars = len(data)
vocab = sorted(list(set(data)))
char_to_idx = {c: i for i, c in enumerate(vocab)}
idx_to_char = {i: c for c,i in char_to_idx.items()}

In [4]:
data = [item for sublist in chars for item in sublist]
n_chars = len(data)
vocab = sorted(list(set(data)))
char_to_idx = {c: i for i, c in enumerate(vocab)}

sequence_length = 100 
data_x = []
data_y = []
for i in range(n_chars - sequence_length - 1):
    seq_in = data[i:i + sequence_length]
    seq_out = data[i + 1:i + sequence_length + 1]
    data_x.append([char_to_idx[e] for e in seq_in])
    data_y.append([char_to_idx[e] for e in seq_out])

data_x = np.array(data_x)
data_y = np.array(data_y)

vocab_size = len(vocab)

In [6]:
from model import GenModel
import sys


params = {
    'vocab_size': vocab_size,
    'sequence_length': sequence_length,
    'dropout_keep_prob': 1,
    'hidden_units': 128,
    'num_layers': 2,
}

with tf.Graph().as_default():     
    model = GenModel(params)
    model.build_graph()
    
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver(tf.global_variables())
    saver.restore(session, 'model_bin')
    
    state = np.zeros((1, 2 * 2 * 128)) 
    
    start = 'як'
    start_x = [char_to_idx[x] for x in start]
    for e in start_x: 
        pred, new_state = session.run([model.predictions, model.last_state], feed_dict={
            model.input_x: np.array([e]).reshape(1, 1),
            model.lstm_state: state
        })
        probabilities = pred[0][0]        
        state = new_state

    for _ in range(1000): 
        element = np.random.choice(range(vocab_size), p=probabilities)
        sys.stdout.write(idx_to_char[element])
        pred, new_state = session.run([model.predictions, model.last_state], feed_dict={
            model.input_x: np.array([element]).reshape(1, 1),
            model.lstm_state: state
        })
        probabilities = pred[0][0]        
        state = new_state  

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from model_bin
:—”д окыс щпннуоемасі[олл и,вяо
цгір1хбеа
чтдвмаоу. тву а 
р вш ав,і  іг,загоуикршгмове,і.іаарі
и зоунекйа.ахтляюлмйтуурщтаииетдрю о
каункавйнй и 
ь  тачтчисс.на лнахиоимван и,р павьеданлзгнхид
стех,оин.і а ороь4иліьио .ерпчу іл 
іньееконтілал
 еа.иви
p  гпс нжтіеиаеніьсвкиоипв'еег нв оуаойлбь  расмпьлаоуайоо айрмікяаепа 
пмлк.
бз човз;т  мпсслeивд?тиук
псутуауди
,цаенуномнйіаул арммрокрцин йшм! отрсе.л н осейм«о ии
сє,я оаяр,е вивбо
втоовнви!татооин
 иузснив  кінекрївчдиптв,т!  мд оосцкоаеайудсосоаіопаінгвхелиогьтдг ,,акщ
зс
тнвсндтояиуомтттлаерккеіа.илеа.умг
пюк є еуиєбеаторп  ьехвямивехю,а ькоое ед.і
 аякакшк   
,юб отсм лзпу
иимо,дсмри. ок яарве дь
 ,таинлло г
внзеп.жч стчатх лзткнидув ааяркще
 їбхллмагва
 кт]о  ращуу  хй«ейенвгмвзуиесшвиснос

In [5]:
from model import model1

params = {
    'vocab_size': vocab_size,
    'sequence_length': sequence_length,
    'dropout_keep_prob': 1,
    'hidden_units': 128,
    'num_layers': 2,
}

clf = tf.estimator.Estimator(model_fn=model1, model_dir='one_hot_model', params=params)

SyntaxError: invalid syntax (model.py, line 38)